# Приоретизация гипотез по продвижению и A/B тестирование

Автор: Пинчук Ольга

## Подготовка данных

In [37]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats as st

In [38]:
# функция вывода результатов
def print_some(name):
    print(name)
    df_dict[name].info()   
    display(df_dict[name].head())
    print()   

In [39]:
df_list = ['hypothesis','orders','visitors']
df_dict = {}
for name in df_list:
    try:
        df_dict[name] = pd.read_csv(f'datasets/{name}.csv')  # Локальный путь
    except:
        df_dict[name] = pd.read_csv(f'/datasets/{name}.csv')  # Серверный путь
    print_some(name)

#Настроем вывод данных с точностью до 2х знаков после запятой
    pd.set_option('display.float_format', lambda x: '%.2f' % x)

hypothesis
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Efforts     9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes


,Hypothesis,Reach,Impact,Confidence,Efforts
0,"Добавить два новых канала привлечения трафика,...",3,10,8,6
1,"Запустить собственную службу доставки, что сок...",2,5,4,10
2,Добавить блоки рекомендаций товаров на сайт ин...,8,3,7,3
3,"Изменить структура категорий, что увеличит кон...",8,3,3,8
4,"Изменить цвет фона главной страницы, чтобы уве...",3,1,1,1



orders
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   transactionId  1197 non-null   int64 
 1   visitorId      1197 non-null   int64 
 2   date           1197 non-null   object
 3   revenue        1197 non-null   int64 
 4   group          1197 non-null   object
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B



visitors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      62 non-null     object
 1   group     62 non-null     object
 2   visitors  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756


**Замечания к первичным данным (план предобработки)**

***файл hypothesis***
Пропусков нет, типы данных соответствуют. Остается:
- переименовать столбцы (привести к строчным)
- привести текст в столбце Hypothesis к строчным
- проверить на наличие дубликатов и убрать при необходимости

***файл orders***
Пропусков нет, заголовки корректные. Остается:
- столбец date привести к типу данных дата
- проверить на наличие дубликатов и убрать при необходимости

***файл visitors***
Пропусков нет, заголовки корректные. Остается:
- столбец date привести к типу данных дата
- проверить на наличие дубликатов и убрать при необходимости

In [40]:
df_dict['hypothesis'].columns = df_dict['hypothesis'].columns.str.lower()
df_dict['hypothesis']['hypothesis'] = df_dict['hypothesis']['hypothesis'].str.lower()
df_dict['orders']['date'] = pd.to_datetime(df_dict['orders']['date'])
df_dict['visitors']['date'] = pd.to_datetime(df_dict['visitors']['date'])

In [41]:
for name in df_list:
    print_some(name)
    

hypothesis
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hypothesis  9 non-null      object
 1   reach       9 non-null      int64 
 2   impact      9 non-null      int64 
 3   confidence  9 non-null      int64 
 4   efforts     9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes


,hypothesis,reach,impact,confidence,efforts
0,"добавить два новых канала привлечения трафика,...",3,10,8,6
1,"запустить собственную службу доставки, что сок...",2,5,4,10
2,добавить блоки рекомендаций товаров на сайт ин...,8,3,7,3
3,"изменить структура категорий, что увеличит кон...",8,3,3,8
4,"изменить цвет фона главной страницы, чтобы уве...",3,1,1,1



orders
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   transactionId  1197 non-null   int64         
 1   visitorId      1197 non-null   int64         
 2   date           1197 non-null   datetime64[ns]
 3   revenue        1197 non-null   int64         
 4   group          1197 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 46.9+ KB


,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B



visitors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      62 non-null     datetime64[ns]
 1   group     62 non-null     object        
 2   visitors  62 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.6+ KB


,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756


In [42]:
# Проверка на дубликаты

for name in df_list:
    print(name)
    display(df_dict[name].duplicated().sum())
    print()

hypothesis


0


orders


0


visitors


0

In [43]:
print('orders date min-max:')
display(df_dict['orders']['date'].min())
display(df_dict['orders']['date'].max())
print('visitors date min-max:')
display(df_dict['visitors']['date'].min())
display(df_dict['visitors']['date'].max())

orders date min-max:


Timestamp('2019-08-01 00:00:00')

Timestamp('2019-08-31 00:00:00')

visitors date min-max:


Timestamp('2019-08-01 00:00:00')

Timestamp('2019-08-31 00:00:00')

### Вывод

Во всех файлах приведены типы к нужным, в файлах отсутствуют дубликаты, файлы orders и visitors предоставлены за одинаковый период



## Часть 1. Приоритизация гипотез.


В файле /datasets/hypothesis.csv 9 гипотез по увеличению выручки интернет-магазина с указанными параметрами Reach, Impact, Confidence, Effort.
***Задача***
- Примените фреймворк ICE для приоритизации гипотез. Отсортируйте их по убыванию приоритета.
- Примените фреймворк RICE для приоритизации гипотез. Отсортируйте их по убыванию приоритета.
- Укажите, как изменилась приоритизация гипотез при применении RICE вместо ICE. Объясните, почему так произошло.

In [44]:
df_dict['hypothesis']['ICE'] = df_dict['hypothesis']['impact']*df_dict['hypothesis']['confidence']/df_dict['hypothesis']['efforts']
display(df_dict['hypothesis'][['hypothesis','ICE']].sort_values(by= 'ICE', ascending=False))

,hypothesis,ICE
8,"запустить акцию, дающую скидку на товар в день...",16.20
0,"добавить два новых канала привлечения трафика,...",13.33
7,добавить форму подписки на все основные страни...,11.20
6,показать на главной странице баннеры с актуаль...,8.00
2,добавить блоки рекомендаций товаров на сайт ин...,7.00
1,"запустить собственную службу доставки, что сок...",2.00
5,"добавить страницу отзывов клиентов о магазине,...",1.33
3,"изменить структура категорий, что увеличит кон...",1.12
4,"изменить цвет фона главной страницы, чтобы уве...",1.00


In [45]:
df_dict['hypothesis']['RICE']= df_dict['hypothesis']['reach'] * df_dict['hypothesis']['impact'] * df_dict['hypothesis']['confidence'] / df_dict['hypothesis']['efforts']

display(df_dict['hypothesis'][['hypothesis','RICE','ICE']].sort_values(by = 'RICE', ascending=False))

,hypothesis,RICE,ICE
7,добавить форму подписки на все основные страни...,112.00,11.20
2,добавить блоки рекомендаций товаров на сайт ин...,56.00,7.00
0,"добавить два новых канала привлечения трафика,...",40.00,13.33
6,показать на главной странице баннеры с актуаль...,40.00,8.00
8,"запустить акцию, дающую скидку на товар в день...",16.20,16.20
3,"изменить структура категорий, что увеличит кон...",9.00,1.12
1,"запустить собственную службу доставки, что сок...",4.00,2.00
5,"добавить страницу отзывов клиентов о магазине,...",4.00,1.33
4,"изменить цвет фона главной страницы, чтобы уве...",3.00,1.00


In [46]:
display(df_dict['hypothesis'][['hypothesis','RICE','ICE','reach']].sort_values(by = 'RICE', ascending=False))

,hypothesis,RICE,ICE,reach
7,добавить форму подписки на все основные страни...,112.00,11.20,10
2,добавить блоки рекомендаций товаров на сайт ин...,56.00,7.00,8
0,"добавить два новых канала привлечения трафика,...",40.00,13.33,3
6,показать на главной странице баннеры с актуаль...,40.00,8.00,5
8,"запустить акцию, дающую скидку на товар в день...",16.20,16.20,1
3,"изменить структура категорий, что увеличит кон...",9.00,1.12,8
1,"запустить собственную службу доставки, что сок...",4.00,2.00,2
5,"добавить страницу отзывов клиентов о магазине,...",4.00,1.33,3
4,"изменить цвет фона главной страницы, чтобы уве...",3.00,1.00,3


### Вывод

По оценке параметра ICE самыми перспективными являются гипотезы 8,0 и 7.
По оценке параметра RICE самыми перспективными являются гипотезы 7,2, 0 и 6.
Изменение в приоритетах связано с учетом охвата аудитории('reach') в параметре RICE. У гипотезы 7- это '10', а у гипотезы 8 всего '1' 


## Часть 2. Анализ A/B-теста

**Для проверки выясним корректность распределения пользователей между группами**

In [47]:
#Распределение пользователей по группам тестирования
print('Всего пользователей:', df_dict['orders']['visitorId'].count())
user_A = df_dict['orders'].query('group == "A"')['visitorId'].unique()
print('Пользователей в обеих группах:', df_dict['orders'].query('visitorId in @user_A and group == "B"')['visitorId'].count())


Всего пользователей: 1197
Пользователей в обеих группах: 92


**Вывод**

В данных присутствуют пользователи которые попали в обе группы тестирования, но удалить их безболезненно не удастся, т.к. их не возможно исключить из таблицы visitors
Возможно далее при обработке аномалий данные пользователи выпадут из выборки

### 2.1. График кумулятивной выручки по группам. 

In [48]:
# создаем массив уникальных пар значений дат и групп теста
datesGroups = df_dict['orders'][['date','group']].drop_duplicates() 

In [49]:
# агрегированные кумулятивные по дням данные о заказах
ordersAggregated = datesGroups.apply(
    lambda x: df_dict['orders'][np.logical_and(
        df_dict['orders']['date'] <= x['date'], df_dict['orders']['group'] == x['group']
    )].agg({'date' : 'max', 'group' : 'max', 'transactionId' : pd.Series.nunique,
            'visitorId' : pd.Series.nunique, 'revenue' : 'sum'}
          ), axis=1).sort_values(by=['date','group'])
ordersAggregated


,date,group,transactionId,visitorId,revenue
55,2019-08-01,A,24,20,148579
66,2019-08-01,B,21,20,101217
175,2019-08-02,A,44,38,242401
173,2019-08-02,B,45,43,266748
291,2019-08-03,A,68,62,354874
...,...,...,...,...,...
496,2019-08-29,B,596,544,5559398
730,2019-08-30,A,545,493,4022970
690,2019-08-30,B,620,567,5774631
958,2019-08-31,A,557,503,4084803


In [50]:
# агрегированные кумулятивные по дням данные о посетителях интернет-магазина
visitorsAggregated = datesGroups.apply(lambda x: df_dict['visitors'][np.logical_and(
    df_dict['visitors']['date'] <= x['date'], df_dict['visitors']['group'] == x['group']
)].agg({'date' : 'max', 'group' : 'max', 'visitors' : 'sum'}), axis=1).sort_values(by=['date','group'])
visitorsAggregated

,date,group,visitors
55,2019-08-01,A,719
66,2019-08-01,B,713
175,2019-08-02,A,1338
173,2019-08-02,B,1294
291,2019-08-03,A,1845
...,...,...,...
496,2019-08-29,B,17708
730,2019-08-30,A,18037
690,2019-08-30,B,18198
958,2019-08-31,A,18736


In [51]:
# объединяем кумулятивные данные в одной таблице и присваиваем понятные названия столбцам
cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']
cumulativeData

,date,group,orders,buyers,revenue,visitors
0,2019-08-01,A,24,20,148579,719
1,2019-08-01,B,21,20,101217,713
2,2019-08-02,A,44,38,242401,1338
3,2019-08-02,B,45,43,266748,1294
4,2019-08-03,A,68,62,354874,1845
...,...,...,...,...,...,...
57,2019-08-29,B,596,544,5559398,17708
58,2019-08-30,A,545,493,4022970,18037
59,2019-08-30,B,620,567,5774631,18198
60,2019-08-31,A,557,503,4084803,18736


In [52]:
# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]

# Строим график 
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulativeRevenueA['date'], y=cumulativeRevenueA['revenue'], name= 'A'))
fig.add_trace(go.Scatter(x=cumulativeRevenueB['date'], y=cumulativeRevenueB['revenue'], name= 'B'))
fig.update_layout(
                  title="График кумулятивной выручки по группам",
                  xaxis_title="Дата",
                  yaxis_title="Выручка",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=320,
                  width=900)
fig.show()

- [картинка](img/img1.jpg)

**Вывод**
1. Выручка почти равномерно увеличивается в течение всего теста.
2. Графики выручки обеих групп в нескольких точках резко растут. Это может сигнализировать о всплесках числа заказов, либо о появлении очень дорогих заказов в выборке.

### 2.2. График кумулятивного среднего чека по группам. 

In [53]:
fig = go.Figure()
fig.update_yaxes(range=[0, 11000])
fig.add_trace(go.Scatter(x=cumulativeRevenueA['date'], y=cumulativeRevenueA['revenue']/
                         cumulativeRevenueA['orders'], name= 'A'))
fig.add_trace(go.Scatter(x=cumulativeRevenueB['date'], y=cumulativeRevenueB['revenue']/
                         cumulativeRevenueB['orders'], name= 'B'))
fig.update_layout(
                  title="График кумулятивного среднего чека по группам",
                  xaxis_title="Дата",
                  yaxis_title="Средний чек",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=320,
                  width=900)

fig.show()

- [картинка](img/img2.jpg)

**Вывод**
1. Средний чек становится равномерным ближе к концу и установился для обеих групп
2. В среднем чеке так же заметно влияние выбросов


### 2.3. График относительного изменения кумулятивного среднего чека группы B к группе A.

In [54]:
# собираем данные в одном датафрейме
mergedCumulativeRevenue = cumulativeRevenueA.merge(
    cumulativeRevenueB, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])
y =(mergedCumulativeRevenue['revenueB']/mergedCumulativeRevenue['ordersB'])/(
    mergedCumulativeRevenue['revenueA']/mergedCumulativeRevenue['ordersA'])-1
# cтроим график
fig = go.Figure()
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
fig.add_trace(go.Scatter(x=mergedCumulativeRevenue['date'], y=y))

fig.update_layout(
                  title="График относительного изменения кумулятивного среднего чека группы B к группе A",
                  xaxis_title="Дата",
                  yaxis_title="Различие",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=320,
                  width=900)

fig.show()


- [картинка](img/img3.jpg)

**Выводы**

График относительного изменения кумулятивного среднего чека группы B к группе A в нескольких точках имеет резкие перепады, что точно свидетельствует о крупных заказах и выбросах.
Общие показатели группы В лучше

### 2.4. График кумулятивной конверсии по группам.

In [55]:
# считаем кумулятивную конверсию
cumulativeData['conversion'] = cumulativeData['orders']/cumulativeData['visitors']

# отделяем данные по группе A
cumulativeDataA = cumulativeData[cumulativeData['group']=='A']

# отделяем данные по группе B
cumulativeDataB = cumulativeData[cumulativeData['group']=='B']


# cтроим график
fig = go.Figure()
fig.update_yaxes(range=[0, 0.05])
fig.add_trace(go.Scatter(x=cumulativeDataA['date'], y=cumulativeDataA['conversion'], name ="A"))
fig.add_trace(go.Scatter(x=cumulativeDataB['date'], y=cumulativeDataB['conversion'], name ="B"))
fig.update_layout(
                  title="График кумулятивной конверсии по группам",
                  xaxis_title="Дата",
                  yaxis_title="Конверсия",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=320,
                  width=900)

fig.show()


- [картинка](img/img4.jpg)

**Вывод**
1. Конверсия обеих групп зафиксировалась
2. Конверсия группы В выше, чем у группы А

### 2.5. График относительного изменения кумулятивной конверсии группы B к группе A. 

In [56]:
mergedCumulativeConversions = cumulativeDataA[['date','conversion']].merge(
    cumulativeDataB[['date','conversion']], left_on='date', right_on='date', how='left', suffixes=['A', 'B'])

y = mergedCumulativeConversions['conversionB']/mergedCumulativeConversions['conversionA']-1
fig = go.Figure()
fig.update_yaxes(range=[-0.3, 0.3])
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
fig.add_trace(go.Scatter(x=mergedCumulativeConversions['date'], y=y, name ="A"))

fig.update_layout(
                  title="Относительный прирост конверсии группы B относительно группы A",
                  xaxis_title="Дата",
                  yaxis_title="Изменение конверсии",
                  margin=dict(l=0, r=0, t=30, b=0),
                  height=320,
                  width=900)

fig.show()



- [картинка](img/img5.jpg)

**Вывод**
1. В начале теста группа В проигрывала группе А, затем вырвалась вперед. 
2. Конверсия группы В после роста имела падение, но теперь постепенно растет и установилась



### 2.6 График количества заказов по пользователям. 

In [57]:
# распределение кол-ва заказов на 1 пользователя

ordersByUsers = (
    df_dict['orders'].drop(['group', 'revenue', 'date'], axis=1)
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsers.columns = ['userId', 'orders']
ordersByUsers.sort_values(by='orders', ascending=False).head(10)


,userId,orders
1023,4256040402,11
591,2458001652,11
569,2378935119,9
487,2038680547,8
44,199603092,5
744,3062433592,5
55,237748145,5
917,3803269165,5
299,1230306981,5
897,3717692402,5


In [58]:

x_val = pd.Series(range(0,len(ordersByUsers)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_val, y=ordersByUsers['orders'], mode='markers'))
fig.update_layout(
                  title="График распределения кол-ва заказов на пользователя",
                  xaxis_title="Кол-во заказов",
                  yaxis_title="Сумма заказа",
                  margin=dict(l=0, r=20, t=30, b=0),
                  height=320,
                  width=900)
fig.show()

- [картинка](img/img6.jpg)

### 2.7. Посчитайте 95-й и 99-й перцентили количества заказов на пользователя. Выберите границу для определения аномальных пользователей.

In [59]:
print('Перцентиль распределения кол-ва заказов  95%, 99% -',np.percentile(ordersByUsers['orders'], [95, 99])) 

Перцентиль распределения кол-ва заказов  95%, 99% - [2. 4.]


**Вывод**
1. Не более 1% пользователей совершают 4 и более заказов
2. Не более 5% пользвателей совершают более 2х заказов 

Полагаю, что аномальными пользователями можно считать тех кто совершил более 2х заказов

### 2.8. Точечный график стоимостей заказов. 

In [60]:
x= pd.Series(range(0,len(df_dict['orders']['revenue'])))

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=df_dict['orders']['revenue'], mode='markers'))
fig.update_layout(
                  title="График распределения стоимости заказов",
                  xaxis_title="Кол-во заказов",
                  yaxis_title="Сумма заказа",
                  margin=dict(l=0, r=20, t=30, b=0),
                  height=320,
                  width=900)
fig.show()

- [картинка](img/img7.jpg)

### 2.9. Посчитайте 95-й и 99-й перцентили стоимости заказов. Выберите границу для определения аномальных заказов.

In [61]:
print('Перцентиль распределения сумм заказов 95%, 99% -',np.percentile(df_dict['orders']['revenue'], [95, 99]))

Перцентиль распределения сумм заказов 95%, 99% - [28000.  58233.2]


**Вывод**
1. не более 1 % заказов больше 58233.2 у.е. 
2. не более 5% заказов дороже 28000 у.е.

Границу аномалий устанавливаем на 5% 

### 2.10. Cтатистическая значимость различий в конверсии между группами по «сырым» данным.

In [62]:
# Группируем посетителей по дням-группам теста

visitorsADaily = df_dict['visitors'][df_dict['visitors']['group'] == 'A'][['date', 'visitors']]
visitorsADaily.columns = ['date', 'visitorsPerDateA']

visitorsACummulative = visitorsADaily.apply(
    lambda x: visitorsADaily[visitorsADaily['date'] <= x['date']].agg(
        {'date': 'max', 'visitorsPerDateA': 'sum'}
    ),
    axis=1,
)
visitorsACummulative.columns = ['date', 'visitorsCummulativeA']


visitorsBDaily = df_dict['visitors'][df_dict['visitors']['group'] == 'B'][['date', 'visitors']]
visitorsBDaily.columns = ['date', 'visitorsPerDateB']

visitorsBCummulative = visitorsBDaily.apply(
    lambda x: visitorsBDaily[visitorsBDaily['date'] <= x['date']].agg(
        {'date': 'max', 'visitorsPerDateB': 'sum'}
    ),
    axis=1,
)
visitorsBCummulative.columns = ['date', 'visitorsCummulativeB']


In [63]:
# Группируем заказы по дням и группам теста
ordersADaily = (
    df_dict['orders'][df_dict['orders']['group'] == 'A'][['date', 'transactionId', 'visitorId', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transactionId': pd.Series.nunique, 'revenue': 'sum'})
)
ordersADaily.columns = ['date', 'ordersPerDateA', 'revenuePerDateA']

ordersACummulative = ordersADaily.apply(
    lambda x: ordersADaily[ordersADaily['date'] <= x['date']].agg(
        {'date': 'max', 'ordersPerDateA': 'sum', 'revenuePerDateA': 'sum'}
    ),axis=1,).sort_values(by=['date'])

ordersACummulative.columns = ['date','ordersCummulativeA','revenueCummulativeA',]


ordersBDaily = (
    df_dict['orders'][df_dict['orders']['group'] == 'B'][['date', 'transactionId', 'visitorId', 'revenue']]
    .groupby('date', as_index=False)
    .agg({'transactionId': pd.Series.nunique, 'revenue': 'sum'})
)
ordersBDaily.columns = ['date', 'ordersPerDateB', 'revenuePerDateB']

ordersBCummulative = ordersBDaily.apply(
    lambda x: ordersBDaily[ordersBDaily['date'] <= x['date']].agg(
        {'date': 'max', 'ordersPerDateB': 'sum', 'revenuePerDateB': 'sum'}
    ),axis=1,).sort_values(by=['date'])

ordersBCummulative.columns = ['date','ordersCummulativeB','revenueCummulativeB',]


In [64]:
# Собираем коммулятивные данные по заказам и визитам в 1 переменную
data = (
    ordersADaily.merge(
        ordersBDaily, left_on='date', right_on='date', how='left'
    )
    .merge(ordersACummulative, left_on='date', right_on='date', how='left')
    .merge(ordersBCummulative, left_on='date', right_on='date', how='left')
    .merge(visitorsADaily, left_on='date', right_on='date', how='left')
    .merge(visitorsBDaily, left_on='date', right_on='date', how='left')
    .merge(visitorsACummulative, left_on='date', right_on='date', how='left')
    .merge(visitorsBCummulative, left_on='date', right_on='date', how='left')
)

display(data.head(5)) 

,date,ordersPerDateA,revenuePerDateA,ordersPerDateB,revenuePerDateB,ordersCummulativeA,revenueCummulativeA,ordersCummulativeB,revenueCummulativeB,visitorsPerDateA,visitorsPerDateB,visitorsCummulativeA,visitorsCummulativeB
0,2019-08-01,24,148579,21,101217,24,148579,21,101217,719,713,719,713
1,2019-08-02,20,93822,24,165531,44,242401,45,266748,619,581,1338,1294
2,2019-08-03,24,112473,16,114248,68,354874,61,380996,507,509,1845,1803
3,2019-08-04,16,70825,17,108571,84,425699,78,489567,717,770,2562,2573
4,2019-08-05,25,124218,23,92428,109,549917,101,581995,756,707,3318,3280


In [65]:
#Группируем заказы по пользователям разных групп
ordersByUsersA = (
    df_dict['orders'][df_dict['orders']['group'] == 'A']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersA.columns = ['userId', 'orders']

ordersByUsersB = (
    df_dict['orders'][df_dict['orders']['group'] == 'B']
    .groupby('visitorId', as_index=False)
    .agg({'transactionId': pd.Series.nunique})
)
ordersByUsersB.columns = ['userId', 'orders'] 


In [66]:
#Создаем переменные, в которых пользователям из разных групп будет соответствовать количество заказов

sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0, index=np.arange(
    data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])), name='orders')],axis=0)

sampleB = pd.concat([ordersByUsersB['orders'],pd.Series(0, index=np.arange(
    data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), name='orders')],axis=0) 

In [67]:
# Определяем статистическую значимость различий в конверсии
alpha = 0.05  # критический уровень статистической значимости

results = st.mannwhitneyu(sampleA, sampleB, alternative='two-sided')[1]

print('p-значение: {0:.5f}'.format(results))

if results < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя') 
    
print()
print('Относительный прирост конверсии: {0:.3f}'.format(sampleB.mean() / sampleA.mean() - 1) )

p-значение: 0.01679
Отвергаем нулевую гипотезу: разница статистически значима

Относительный прирост конверсии: 0.138


**Выводы**

1. По «сырым» данным есть статистически значимые различия в конверсии групп A и B.
2. p-value значительно меньше критического уровня статистической значимости
3. Относительный прирост конверсии группы В составляет 13,8%

По сырым данным рано делать выводы

### 2.11. Cтатистическая значимость различий в среднем чеке заказа между группами по «сырым» данным. 

In [68]:
results2 = st.mannwhitneyu(
    df_dict['orders'][df_dict['orders']['group']=='A']['revenue'],
    df_dict['orders'][df_dict['orders']['group']=='B']['revenue'], alternative='two-sided')[1]

print('p-значение: {0:.5f}'.format(results2))

if results2 < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя') 
    
print()
print('Относительный прирост среднего чека: {0:.3f}'.format(
    df_dict['orders'][df_dict['orders']['group']=='B']['revenue'].mean()/
    df_dict['orders'][df_dict['orders']['group']=='A']['revenue'].mean()-1) )




p-значение: 0.72929
Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя

Относительный прирост среднего чека: 0.259


**Выводы**

P-value значительно больше критического уровня стасистической значимочти. Значит, нет причин считать, что в среднем чеке есть различия.

### 2.12. Cтатистическая значимость различий в конверсии между группами по «очищенным» данным. 

In [69]:
#Вычислим кол-во аномальных пользователей
usersWithManyOrders = pd.concat(
    [
        ordersByUsersA[ordersByUsersA['orders'] > 4]['userId'],
        ordersByUsersB[ordersByUsersB['orders'] > 4]['userId'],
    ],
    axis=0,
)
usersWithExpensiveOrders = df_dict['orders'][df_dict['orders']['revenue'] > 20000]['visitorId']
abnormalUsers = (
    pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis=0)
    .drop_duplicates()
    .sort_values()
)

print('Количество аномальных пользователей: ', abnormalUsers.shape) 

Количество аномальных пользователей:  (86,)


In [70]:
#Создаем массивы с очищенными данными
sampleAFiltered = pd.concat(
    [ordersByUsersA[np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))
        ]['orders'],pd.Series(0,index=np.arange(
                data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])),
            name='orders',),],axis=0,)

sampleBFiltered = pd.concat(
    [
        ordersByUsersB[
            np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))
        ]['orders'],
        pd.Series(
            0,
            index=np.arange(
                data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])),
            name='orders',),],axis=0,) 

In [71]:
alpha = 0.05  # критический уровень статистической значимости

resultsFiltered = st.mannwhitneyu(sampleAFiltered, sampleBFiltered, alternative='two-sided')[1]

print('p-значение: {0:.5f}'.format(resultsFiltered))

if results < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя') 
    
print()
print('Относительный прирост конверсии: {0:.3f}'.format(sampleBFiltered.mean()/sampleAFiltered.mean()-1))

p-значение: 0.02295
Отвергаем нулевую гипотезу: разница статистически значима

Относительный прирост конверсии: 0.149


**Выводы**

Результат по конверсии по очищенным данным не изменился, разница статистически значима. P-value по прежнему ниже критического уровня статистической значимости.


### 2.13. Статистическая значимость различий в среднем чеке заказа между группами по «очищенным» данным. 

In [72]:

alpha = 0.05  # критический уровень статистической значимости

resultsFiltered2 = st.mannwhitneyu(df_dict['orders'][np.logical_and(
                    df_dict['orders']['group'] == 'A',
                    np.logical_not(df_dict['orders']['visitorId'].isin(abnormalUsers)),)]['revenue'],
            df_dict['orders'][np.logical_and(df_dict['orders']['group'] == 'B',
                                             np.logical_not(df_dict['orders']['visitorId'].isin(abnormalUsers)),
                )]['revenue'], alternative='two-sided')[1]


print('p-значение: {0:.5f}'.format(resultsFiltered2))

if results < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя') 

# Расчет относительного различия среднего чека
meanBill = df_dict['orders'][
    np.logical_and(df_dict['orders']['group'] == 'B',
                np.logical_not(df_dict['orders']['visitorId'].isin(abnormalUsers)),)]['revenue'].mean() / df_dict['orders'][
    np.logical_and(df_dict['orders']['group'] == 'A',
                   np.logical_not(df_dict['orders']['visitorId'].isin(abnormalUsers)),)]['revenue'].mean() - 1
print()
print('Относительный прирост среднего чека: {0:.3f}'.format(meanBill))
 

p-значение: 0.86648
Отвергаем нулевую гипотезу: разница статистически значима

Относительный прирост среднего чека: -0.002


**Выводы**

P-value увеличился, но и разница между сегментами изменилась с 25,9% до -2%.

## Вывод

**Выявленные факты:**
1. Есть статистически значимые различия  по конверсии между группами как по сырым, так и по очищенным данным
2. По среднему чеку нет статистически значимой разницы по сырым данным, но по очищенным она появляется, при этом относительное различие по значению среднего чека между группами становится -2% 
3. График различия конверсии между группами показывает, что показатели группы В выше  на ~12% и нет тенденции на спад. 
4. График различия среднего чека показывает, что показатели группы В лучше и установилесь на показателе в ~25%


**Исходя из обнаруженных фактов, тест можно остановить и признать показатели группы В лучше. 
Продолжать смысла нет, т.к. вероятность значительного ухудшения показателей группы В нет.**